In [1]:
import os
import json
import pandas as pd
import traceback
import dotenv

In [5]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
KEY = os.getenv("OPENAI_API_KEY")

In [13]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = 'gpt-3.5-turbo', temperature=0.5)

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables= ["text", "number", "subject", "tone", "response_json"],
    template= TEMPLATE
)

In [14]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [17]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [19]:
file_path = '/Users/sohailshaik/mcqgen/data.txt'

In [20]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [22]:
#serealizing the python disctionary into JSON

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [23]:
NUMBER=5 
SUBJECT="bitcoin"
TONE="simple"

In [24]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Bitcoin (abbreviation: BTC[a]; sign: ₿) is the first decentralized cryptocurrency. Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger, called a blockchain, without central oversight. Consensus between nodes is achieved using a computationally intensive process based on proof of work, called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.[5]
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.[6] Use of bitcoin as a currency began in 2009,[7] with the release of its open-source implementation.[8]: ch. 1  In 2021, El Salvador adopted it as legal tender.[4] Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account. It is mostly seen as an investmen

In [25]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2337
Prompt Tokens:1882
Completion Tokens:455
Total Cost:0.003733


In [26]:
response

{'text': 'Bitcoin (abbreviation: BTC[a]; sign: ₿) is the first decentralized cryptocurrency. Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger, called a blockchain, without central oversight. Consensus between nodes is achieved using a computationally intensive process based on proof of work, called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.[5]\nBased on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.[6] Use of bitcoin as a currency began in 2009,[7] with the release of its open-source implementation.[8]:\u200ach. 1\u200a In 2021, El Salvador adopted it as legal tender.[4] Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account. It is mostly seen as an investment and has been described by many scholars as an economic bubble.[9] As bitcoin is pseud

In [27]:
quiz=response.get("quiz")

In [29]:
quiz=json.loads(quiz)

In [30]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [31]:
quiz_table_data

[{'MCQ': 'When was the domain name bitcoin.org registered?',
  'Choices': 'a: 2008 | b: 2009 | c: 2010 | d: 2011',
  'Correct': 'a'},
 {'MCQ': 'Who is the unknown person who invented Bitcoin in 2008?',
  'Choices': 'a: David Chaum | b: Satoshi Nakamoto | c: Adam Back | d: Hal Finney',
  'Correct': 'b'},
 {'MCQ': 'Which major company invested $250 million in bitcoin as a treasury reserve asset in 2020?',
  'Choices': 'a: Square, Inc. | b: MassMutual | c: MicroStrategy | d: PayPal',
  'Correct': 'c'},
 {'MCQ': 'What was the headline embedded in the genesis block of the Bitcoin network?',
  'Choices': 'a: Bitcoin: A Peer-to-Peer Electronic Cash System | b: Chancellor on brink of second bailout for banks | c: Bitcoin logos made by Satoshi Nakamoto | d: The Times 03/Jan/2009',
  'Correct': 'b'},
 {'MCQ': "In which year did Bitcoin's market capitalization reach $1 trillion for the first time?",
  'Choices': 'a: 2019 | b: 2020 | c: 2021 | d: 2022',
  'Correct': 'c'}]

In [32]:
quiz=pd.DataFrame(quiz_table_data)

In [33]:
quiz.to_csv(SUBJECT+"_quiz.csv",index=False)